In [12]:
from datetime import datetime
import sys
import time
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
from PIL import Image
import html
from markdownify import markdownify 
import re
import urllib.parse
from io import BytesIO




endpoint_url = "https://query.wikidata.org/sparql"

query = """
SELECT DISTINCT ?item ?itemLabel ?image ?artistLabel (YEAR(?inception) AS ?year) WHERE {
  ?item wdt:P180 wd:Q146;
        wdt:P31 wd:Q3305213;
        wdt:P18 ?image;
        wdt:P571 ?inception;
        wdt:P170 ?artist.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],mul,en". }
}
ORDER BY DESC(?year)
LIMIT 30
"""


def run_query(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setMethod("POST") #this NEEDS to be added to get results (not included in the wikibase python example code)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

def get_img(partOfItem_id):

    print('<script src="https://unpkg.com/masonry-layout@4/dist/masonry.pkgd.min.js"></script>')
    print('<script src="https://unpkg.com/imagesloaded@4/imagesloaded.pkgd.min.js"></script>')
    print("<link rel='stylesheet' href='./masonrydesign.css'>")
    print('<div class="img_grid">')
    q = query

    results = run_query(endpoint_url, q)

#    rows = int(len(results['head']))
#    counter = 0
#     print(f"""
# ::::: {{layout-ncol=2}}
#     """)
    for item in results["results"]["bindings"]:    

      alt='image of a ' + item['itemDescription']['value'] if 'itemDescription' in item else 'image'
      title=item['itemLabel']['value'].replace("'", "&#39;") if 'itemLabel' in item else ''
      year=item['year']['value'] if 'year' in item else ''
      artist=item['artistLabel']['value'] if 'artistLabel' in item else ''
      item_url=item['item']['value'] if 'item' in item else ""
      image_url=item['image']['value'] if 'image' in item else ''

      filename = image_url.split('/')[-1].split('.')
      url_end = re.sub('[^0-9a-zA-Z]+', '_', filename[-2]) 
      image_path = './images/' + url_end + '.' + filename[-1] 

      headers = {
    "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36"
      }


      response = requests.get(image_url, allow_redirects=True, headers=headers)
      image = Image.open(BytesIO(response.content))
      image.thumbnail((500, 500))
      image.save(f'{image_path}')



      openpop = f'openPopover("{image_url}")'
      closepop = f"closePopover('{image_url}')"
      openinfo = f'openInfo("{item_url}")'
      closeinfo = f"closeInfo('{item_url}')"

      print("""
::: {.content-hidden when-format='pdf'}
      """)

      print(f"<div class='info-wd btn-group' id=\"{item_url}\" style='display:none'><a href='{item_url}'><button class='btn btn-light'>{title}</button></a><button type='button' class='btn btn-secondary'>by:  {artist}</button><button type='button' class='btn btn-secondary'>{year}</button></div>")
      print(f"<img title=\'{title}\' class='img_small' src='{image_url}' popovertarget='{image_url}' onclick='{openpop};{openinfo}' alt='{alt}'>")
      #print(f'<input type="image" class="img_small" src="{image_url}" popovertarget="{image_url}" onclick="{onclickopen}" />')
      print(f'<img popover class="img_popover" src="{image_url}" id="{image_url}" onclick="{closepop},{closeinfo}" alt="{alt}" style="display: none;">')
      
      print("""
:::
      """)
      print()
      print("""
::: {.content-visible when-format='pdf'}
      """)

#       print(f"""
# ## {title} ({year}) by {artist}
#       """)



      print(f"![{title} ({year}) by {artist}]({image_path}){{height=10cm,keepaspectratio}}")

#      if counter%4 == 0 and counter != 0:
      print("""                     
```{=latex}
\\clearpage
```
        """)
        
#      counter += 1


      print("""
:::
      """)  
#     print("""
# :::::
#     """)
    print('</div>')


    print("""
<script>
  var grid = document.querySelector('.img_grid');
          
  grid.style.display = 'none';

  // Initialize Masonry, but delay layout until images are loaded
  var msnry = new Masonry(grid, {
    itemSelector: '.img_small',
    percentPosition: true,
  });
  

  // Use imagesLoaded to fix layout issues as images load
  imagesLoaded(grid, function() {
    grid.style.display = 'block';
    msnry.layout();
  });

  function openPopover(imageUrl) {
    var popover = document.getElementById(imageUrl);
    var body = document.querySelector('body');
    // Show the popover
    popover.style.display = 'block';
    body.classList.add("stop-scrolling");
  };
          
  function openInfo(itemUrl) {
    var popover = document.getElementById(itemUrl);
    var body = document.querySelector('body');
    // Show the popover
    popover.style.display = 'flex';
  };

  function closePopover(imageUrl) {
    var popover = document.getElementById(imageUrl);
    var body = document.querySelector('body');
    popover.style.display = 'none';
    body.classList.remove("stop-scrolling");
  };
          
  function closeInfo(itemUrl) {
    var popover = document.getElementById(itemUrl);
    var body = document.querySelector('body');
    popover.style.display = 'none';
  };
          
</script>
          """)





In [13]:
get_img("")

<script src="https://unpkg.com/masonry-layout@4/dist/masonry.pkgd.min.js"></script>
<script src="https://unpkg.com/imagesloaded@4/imagesloaded.pkgd.min.js"></script>
<link rel='stylesheet' href='./masonrydesign.css'>
<div class="img_grid">

::: {.content-hidden when-format='pdf'}
      
<div class='info-wd btn-group' id="http://www.wikidata.org/entity/Q132252693" style='display:none'><a href='http://www.wikidata.org/entity/Q132252693'><button class='btn btn-light'>Cat Nap</button></a><button type='button' class='btn btn-secondary'>by:  Fay Kleinman</button><button type='button' class='btn btn-secondary'>2003</button></div>
<img title='Cat Nap' class='img_small' src='http://commons.wikimedia.org/wiki/Special:FilePath/Cat%20Nap%2C%202003%2C%20oil.jpg' popovertarget='http://commons.wikimedia.org/wiki/Special:FilePath/Cat%20Nap%2C%202003%2C%20oil.jpg' onclick='openPopover("http://commons.wikimedia.org/wiki/Special:FilePath/Cat%20Nap%2C%202003%2C%20oil.jpg");openInfo("http://www.wikidata.or

In [14]:
#get_graph()